<a href="https://colab.research.google.com/github/vsuhas9/StyleTransfer/blob/dev-suhas/Style%20Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [36]:
# Package to handle files
import os

# Image Handling
import numpy as np

# Tensorflow Componenets
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19

# Few Initial Configuration

In [ ]:
result_prefix = "generated"

# Weights of the different loss components
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

# Dimensions of the generated picture.
img_nrows = 600
#img_ncols = int(width * img_nrows / height)

# Download the Transfer Images and Style Images

In [34]:
!rm -rf target_images.zip
!rm -rf target_images
!wget https://raw.githubusercontent.com/vsuhas9/StyleTransfer/dev-suhas/Images/target/target_images.zip
!unzip /content/target_images.zip

!rm -rf style_images.zip
!rm -rf style_images
#!wget https://raw.githubusercontent.com/vsuhas9/StyleTransfer/dev-suhas/Images/style/style_images.zip
#!unzip /content/style_images.zip

--2024-01-18 11:36:12--  https://raw.githubusercontent.com/vsuhas9/StyleTransfer/dev-suhas/Images/target/target_images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16394240 (16M) [application/zip]
Saving to: ‘target_images.zip’

target_images.zip   100%[===================>]  15.63M  --.-KB/s    in 0.1s    

2024-01-18 11:36:12 (129 MB/s) - ‘target_images.zip’ saved [16394240/16394240]

Archive:  /content/target_images.zip
   creating: target_images/
  inflating: target_images/Bench_Event221_SetA.png  
  inflating: target_images/BG.png    
  inflating: target_images/Bushes1_Event221_SetA.png  
  inflating: target_images/Bushes2_Event221_SetA.png  
  inflating: target_images/Bushes3_Event221_SetA.png  
  inflating: target_images/Bushes4_Event221_SetA.p

# Image paths and other componenets

In [41]:
def list_files(folder_path):
    files = []
    for entry in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, entry)):
            files.append( folder_path + entry)
    return files

# Example usage
target_folder_path = '/content/target_images/'
style_folder_path = '/content/target_images/'

try:
  target_images = [keras.preprocessing.image.load_img(img_path) for img_path in list_files(target_folder_path)]
  style_images = [keras.preprocessing.image.load_img(img_path) for img_path in list_files(style_folder_path)]

  print(str(len(target_images)) + " Target Images loaded & " + str(len(style_images)) + " Style Images loaded" )
except:
  print("Error loading the files")

34 Target Images loaded & 34 Style Images loaded

